# Import Libraries

In [1]:
# Pandas and Numpy to format the data
import pandas as pd
import numpy as np

# Keras to build the model
import keras
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.models import Model, model_from_json
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

# Pickle to load the data and save the models
import pickle

Using TensorFlow backend.


# Model Architecture
Model to embed images of clothing

In [2]:
# Initialize the imput as a color image size 64x64 pixels
input_layer = Input(shape=(3,64,64))

# Encoder: Increase dimensionality in convolutions, then reduce to 16x1x1
conv1 = Conv2D(496, (3, 3), activation='relu', padding='same')(input_layer)
pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv1)
conv2 = Conv2D(248, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv2)
conv3 = Conv2D(124, (3, 3), activation='relu', padding='same')(pool2)
pool3 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv3)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool3)
pool4 = MaxPooling2D(pool_size=(2, 2), padding='same')(conv4)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool4)
pool5 = MaxPooling2D((2, 2), padding='same')(conv5)
conv6 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool5)
pool6 = MaxPooling2D((2, 2), padding='same')(conv6)
conv7 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool6)
pool7 = MaxPooling2D((2, 2), padding='same')(conv7)

# Reduced dimensionailty layer that can be taken as an embedding of the original image
encoded = Flatten(name = 'encoded')(pool7)

# Decoder: Reverse the process of the encoder
conv8 = Conv2D(16, (3, 3), activation='relu', padding='same')(pool7) 
up1 = UpSampling2D((2,2))(conv8)
conv9 = Conv2D(16, (3, 3), activation='relu', padding='same')(up1) 
up2 = UpSampling2D((2,2))(conv9)
conv10 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2) 
up3 = UpSampling2D((2,2))(conv10) 
conv11 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
up4 = UpSampling2D((2,2))(conv11)
conv12 = Conv2D(124, (3, 3), activation='relu', padding='same')(up4)
up5 = UpSampling2D((2,2))(conv12) 
conv13 = Conv2D(248, (3, 3), activation='relu', padding='same')(up5)
up6 = UpSampling2D((1, 1))(conv13)
conv14 = Conv2D(496, (3, 3), activation='relu', padding='same')(up6)
up7 = UpSampling2D((2, 2))(conv14)

# Final layer that is the same shape as the input. This is the result that should return the same image as the input
decoded = Conv2D(3, (5, 5), activation='sigmoid', padding='same')(up7)
# Check the output is the same shape as the input
print ('shape of decoded', K.int_shape(decoded))

# Initialize input and output
autoencoder = Model(input_layer, decoded)

# Model that will return the embedding rather than the predicted image, but trained using the autoencoded model
encoder = Model(input_layer, encoded)

shape of decoded (None, 3, 64, 64)


In [3]:
# Save the architextures as strings
json_autoencoder = autoencoder.to_json()
json_encoder = encoder.to_json()

In [4]:
# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

# Run All the Models
Train separate models for 7 different types of clothing that could comprise an outfit

In [5]:
item = ['dress','top','pullover','outerwear','bottom','shoe','bag']

for i in range(1):
    # Establish the models from the jsons
    autoencoder = model_from_json(json_autoencoder)
    encoder = model_from_json(json_encoder)
    
    # Import the images (which are preprocessed and stored in a numpy array of shape (15049,3,64,64))
    path = '../small_image_vectors/' + item[i] + '.pkl'
    with open(path, 'rb') as picklefile: 
        features = pickle.load(picklefile)
    # Standardize the features
    features = features/255
    
    # Fit the Model
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    print('Fitting model '+str(i+1))
    autoencoder.fit(features, features, epochs=400, batch_size=100, validation_split=.15, callbacks=[early_stopping])
    
    # Save the model and and encoder
    autoencoder_name = item[i] + '_model.h5'
    autoencoder.save(autoencoder_name)
    encoder_name = item[i] + 'embedding_model.h5'
    encoder.save(encoder_name)
    
    # Delete the images and models to make room in memory
    del(features)
    del(autoencoder)
    del(encoder)

# Alternate Model Trained on All Articles of Clothing

In [6]:
# Load all the images
with open("../small_image_vectors/dress.pkl", 'rb') as picklefile: 
    dress_features = pickle.load(picklefile)

with open("../small_image_vectors/top.pkl", 'rb') as picklefile: 
    top_features = pickle.load(picklefile)

with open("../small_image_vectors/pullover.pkl", 'rb') as picklefile: 
    pullover_features = pickle.load(picklefile)
    
with open("../small_image_vectors/outerwear.pkl", 'rb') as picklefile: 
    outerwear_features = pickle.load(picklefile)
    
with open("../small_image_vectors/bottom.pkl", 'rb') as picklefile: 
    bottom_features = pickle.load(picklefile)
    
with open("../small_image_vectors/shoe.pkl", 'rb') as picklefile: 
    shoe_features = pickle.load(picklefile)
    
with open("../small_image_vectors/bag.pkl", 'rb') as picklefile: 
    bag_features = pickle.load(picklefile)

# Concatenate all the images together
features = np.concatenate((dress_features,
                           top_features,
                           pullover_features,
                           outerwear_features,
                           bottom_features,
                           shoe_features,
                           bag_features
                          ), axis=0)

# Delete the individual features for space
del(dress_features,
    top_features,
    pullover_features,
    outerwear_features,
    bottom_features,
    shoe_features,
    bag_features
   )

# Standardize the features
features = features/255

In [7]:
# Establish the models from the jsons
autoencoder = model_from_json(json_autoencoder)
encoder = model_from_json(json_encoder)

In [8]:
# Fit the Model
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.fit(features, features, epochs=400, batch_size=100, validation_split=.15, callbacks=[early_stopping])

In [9]:
# Save the model and and encoder
autoencoder.save('all_model.h5')
encoder.save('all_embedding_model.h5')

In [10]:
# Delete the images and models to make room in memory
del(features)
del(autoencoder)
del(encoder)